In [1]:
%matplotlib widget

import logging
import time

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from sqlalchemy import (
    bindparam, 
    func
)

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas import coverages
from arpav_ppcv.schemas.coverages import CoverageInternal
from arpav_ppcv.schemas import observations

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [8]:
cov_id = "tdd_annual_seasonal-annual-historical-absolute-tdd-S03"
cov = CoverageInternal(
    identifier=cov_id,
    configuration=db.get_coverage_configuration_by_coverage_identifier(session, cov_id)
)

In [9]:
cov.configuration.coverage_id_pattern

'{name}-{aggregation_period}-{collection}-{measure}-{observation_variable}-{observation_year_period}'

In [10]:
cov.configuration.get_thredds_url_fragment(cov.identifier)

'cline_yr/TDd_S03_1992-202[34]_py85.nc'

In [11]:
from arpav_ppcv.thredds import crawler

In [12]:
await crawler.find_thredds_dataset_url_fragment(
    httpx.AsyncClient(),
    cov.configuration.get_thredds_url_fragment(cov.identifier),
    "https://thredds.arpa.veneto.it/thredds"
)

'cline_yr/TDd_S03_1992-2023_py85.nc'

In [12]:
import fnmatch
from xml.etree import ElementTree as etree

In [6]:
settings.thredds_server.base_url

'http://thredds:8080/thredds'

In [10]:
response = http_client.get("https://thredds.arpa.veneto.it/thredds/catalog/cline_yr/catalog.xml")

In [33]:
response.content

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<catalog xmlns="http://www.unidata.ucar.edu/namespaces/thredds/InvCatalog/v1.0" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.2">\r\n  <service name="all" serviceType="compound" base="">\r\n    <service name="odap" serviceType="OpenDAP" base="/thredds/dodsC/" />\r\n    <service name="dap4" serviceType="DAP4" base="/thredds/dap4/" />\r\n    <service name="http" serviceType="HTTPServer" base="/thredds/fileServer/" />\r\n    <service name="wcs" serviceType="WCS" base="/thredds/wcs/" />\r\n    <service name="wms" serviceType="WMS" base="/thredds/wms/" />\r\n    <service name="ncssGrid" serviceType="NetcdfSubset" base="/thredds/ncss/grid/" />\r\n  </service>\r\n  <dataset name="CLINE annuali">\r\n    <property name="DatasetScan" value="true" />\r\n    <metadata inherited="true">\r\n      <serviceName>all</serviceName>\r\n      <authority>Arpav</authority>\r\n      <dataType>Grid</dataType>\r\n    </metadata>\r\n    <dataset name="CDD_jrc_A

In [15]:
root = etree.fromstring(response.content)

In [51]:
etree.fromstring('{"a": "b"}')

ParseError: not well-formed (invalid token): line 1, column 0 (<string>)

In [19]:
ns = {
    "thredds": "http://www.unidata.ucar.edu/namespaces/thredds/InvCatalog/v1.0"
}

In [23]:
f"{{{ns['thredds']}}}/service"

'{http://www.unidata.ucar.edu/namespaces/thredds/InvCatalog/v1.0}/service'

In [32]:
f".//{{{ns['thredds']}}}dataset"

'.//{http://www.unidata.ucar.edu/namespaces/thredds/InvCatalog/v1.0}dataset'

In [50]:
fnmatch.filter((ds_el.get("name", "") for ds_el in root.findall(f".//{{{ns['thredds']}}}dataset")), "PRCPTOT_M01_1992-202[34]_py85.nc")

['PRCPTOT_M01_1992-2024_py85.nc']

In [39]:
for dataset_el in root.findall(f".//{{{ns['thredds']}}}dataset"):
    if (name := dataset_el.get("name", "")).endswith(".nc"):
        print(dataset_el.get("name"))

CDD_jrc_A00_1992-2023_py85.nc
FD_A00_1992-2023_py85.nc
HDD_it_A00_1992-2023_py85.nc
PRCPTOT_A00_1992-2023_py85.nc
PRCPTOT_M01_1992-2024_py85.nc
PRCPTOT_M02_1992-2024_py85.nc
PRCPTOT_M03_1992-2024_py85.nc
PRCPTOT_M04_1992-2024_py85.nc
PRCPTOT_M05_1992-2024_py85.nc
PRCPTOT_M06_1991-2024_py85.nc
PRCPTOT_M07_1992-2024_py85.nc
PRCPTOT_M08_1991-2023_py85.nc
PRCPTOT_M09_1991-2023_py85.nc
PRCPTOT_M10_1991-2023_py85.nc
PRCPTOT_M11_1991-2023_py85.nc
PRCPTOT_M12_1991-2023_py85.nc
PRCPTOT_S01_1992-2024_py85.nc
PRCPTOT_S02_1992-2024_py85.nc
PRCPTOT_S03_1992-2023_py85.nc
PRCPTOT_S04_1991-2023_py85.nc
SU30_A00_1992-2023_py85.nc
SU30_M05_1992-2024_py85.nc
SU30_M06_1992-2024_py85.nc
SU30_M07_1992-2024_py85.nc
SU30_M08_1992-2023_py85.nc
SU30_M09_1991-2023_py85.nc
SU30_S03_1992-2023_py85.nc
TDd_A00_1992-2023_py85.nc
TDd_M01_1992-2024_py85.nc
TDd_M02_1992-2024_py85.nc
TDd_M03_1992-2024_py85.nc
TDd_M04_1992-2024_py85.nc
TDd_M05_1992-2024_py85.nc
TDd_M06_1992-2024_py85.nc
TDd_M07_1992-2024_py85.nc
TDd_M08_1